<a href="https://colab.research.google.com/github/RafsanJany-44/High_Ram_NoteBook/blob/main/RandomForest_Experiment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [20]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [21]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)

  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from joblib import Parallel, delayed
from google.colab import files
import joblib
  

def save_model(model,model_name):
 joblib.dump(model, model_name)
  
def load_model(model_name):
  return joblib.load(model_name)

In [23]:
import numpy as np
import pandas as pd
from tqdm import tqdm

dataset=pd.read_csv("https://raw.githubusercontent.com/RafsanJany-44/Research_5_Sleep_Stages_Classification/main/DATA/EEG_Five_Stages_HMC.csv")


target="Sleep_Stage"

classes = np.array(sorted(list(set(dataset[target]))))

In [24]:
dataset.head()

,Sleep_Stage,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,Spectral Edge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,MeanF_Beta_F4,Spectral Edge_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,Spectral Edge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,Spectral Edge_Gamma_O2,PeakF_Gamma_O2
0,W,0.00051,8.74146,18.48195,17.48293,8.24195,0.00071,19.73073,29.84585,33.84195,...,0.00724,0.74927,3.87122,1.99805,0.87415,0.000210,35.21561,42.20878,49.95122,49.95122
1,W,0.00040,10.24000,17.48293,16.85854,10.61463,0.00051,17.23317,26.34927,29.34634,...,0.00723,0.49951,3.74634,1.24878,0.49951,0.000095,34.34146,43.33268,49.95122,49.95122
2,W,0.00036,9.74049,17.98244,17.48293,8.74146,0.00047,17.60780,25.47512,28.22244,...,0.00230,0.62439,5.24488,2.12293,0.62439,0.000074,34.59122,45.95512,49.95122,49.95122
3,W,0.00035,10.11512,17.60780,17.10829,9.61561,0.00047,17.35805,26.47415,28.72195,...,0.00091,0.87415,6.61854,3.12195,0.62439,0.000082,32.46829,43.95707,49.95122,49.95122
4,W,0.00033,9.74049,17.60780,17.23317,8.11707,0.00043,17.23317,25.84976,28.09756,...,0.00175,0.49951,5.74439,2.24780,0.49951,0.000077,33.59220,44.83122,49.95122,49.95122


In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108451 entries, 0 to 108450
Data columns (total 76 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Sleep_Stage             108451 non-null  object 
 1   MeanP_Alpha_F4          108451 non-null  float64
 2   MedianF_Alpha_F4        108451 non-null  float64
 3   MeanF_Alpha_F4          108451 non-null  float64
 4   Spectral Edge_Alpha_F4  108451 non-null  float64
 5   PeakF_Alpha_F4          108451 non-null  float64
 6   MeanP_Beta_F4           108451 non-null  float64
 7   MedianF_Beta_F4         108451 non-null  float64
 8   MeanF_Beta_F4           108451 non-null  float64
 9   Spectral Edge_Beta_F4   108451 non-null  float64
 10  PeakF_Beta_F4           108451 non-null  float64
 11  MeanP_Theta_F4          108451 non-null  float64
 12  MedianF_Theta_F4        108451 non-null  float64
 13  MeanF_Theta_F4          108451 non-null  float64
 14  Spectral Edge_Theta_

In [26]:
set(list(dataset[target]))

{'N1', 'N2', 'N3', 'R', 'W'}

In [27]:
frequency = dataset["Sleep_Stage"].value_counts()
print(frequency)

N2    39428
N3    21290
W     19355
R     16465
N1    11913
Name: Sleep_Stage, dtype: int64


In [28]:

from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

In [29]:
set(list(dataset[target])) # alphabet wise

{0, 1, 2, 3, 4}

In [30]:
frequency = dataset["Sleep_Stage"].value_counts()
print(frequency)

1    39428
2    21290
4    19355
3    16465
0    11913
Name: Sleep_Stage, dtype: int64


In [31]:
X = dataset.loc[:, dataset.columns!=target]
y = dataset[target]

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)


In [37]:
new_dataset=balance(X_train,y_train)

X_train = new_dataset.loc[:, new_dataset.columns!=target]
y_train = new_dataset[target]

In [34]:
'''from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=10000
k_range = range (4500,N+1,500)
scores={}
scores_list = []
for k in tqdm(k_range):
  classifier = RandomForestClassifier(n_estimators=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)'''

'from sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import accuracy_score\nfrom tqdm import tqdm\nN=10000\nk_range = range (4500,N+1,500)\nscores={}\nscores_list = []\nfor k in tqdm(k_range):\n  classifier = RandomForestClassifier(n_estimators=k,random_state=0)\n  classifier.fit(X_train, y_train)\n  y_pred=classifier.predict(X_test)\n  scores[k] = accuracy_score(y_test,y_pred)\n  scores_list.append(accuracy_score(y_test,y_pred))\n  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))\n\n\n\n%matplotlib inline\nimport matplotlib.pyplot as plt\n\nplt.figure(figsize = (25,10))\nplt.plot(k_range,scores_list)\nplt.xlabel(\'Value of n_estimators\')\nplt.ylabel (\'Testing Accuracy\')\n\n\n\nprint("The best n_estimators:")\nbest_estimator=list(scores.keys())[scores_list.index(max(scores_list))]\nprint(best_estimator)'

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
N=100
k_range = range (41,N+1)
scores={}
scores_list = []
for k in tqdm(k_range):
  classifier = RandomForestClassifier(max_depth=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best Depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)

  2%|▏         | 1/60 [02:27<2:25:27, 147.92s/it]

41/100 round completed......................... Accurecy: 0.7851182518095062


  3%|▎         | 2/60 [04:55<2:22:42, 147.62s/it]

42/100 round completed......................... Accurecy: 0.7848416393896086


  5%|▌         | 3/60 [07:24<2:20:43, 148.13s/it]

...

59/100 round completed......................... Accurecy: 0.7857175787192845


 33%|███▎      | 20/60 [49:16<1:38:16, 147.40s/it]

60/100 round completed......................... Accurecy: 0.7857175787192845


 33%|███▎      | 20/60 [51:42<1:43:24, 155.12s/it]


KeyboardInterrupt: ignored

In [ ]:
'''from sklearn.ensemble import RandomForestClassifier
rf_estimator = RandomForestClassifier(n_estimators=best_estimator,random_state=0)
rf_estimator.fit(X_train, y_train)
y_pred=rf_estimator.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

save_model(rf_estimator,"Best_est_"+str(rf_estimator))
#files.download("max_depth_"+str(rf_estimator)) '''

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_depth = RandomForestClassifier(max_depth=best_depth,random_state=0)
rf_depth.fit(X_train, y_train)
y_pred=rf_depth.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

save_model(rf_depth,"max_depth_"+str(rf_depth))
#files.download("max_depth_"+str(rf_depth)) 

In [ ]:
'''from sklearn.ensemble import RandomForestClassifier
rf_all = RandomForestClassifier(n_estimators=450,max_depth=39,random_state=0)
rf_all.fit(X_train, y_train)
y_pred=rf_all.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

save_model(rf_all,"All_"+str(rf_all))
#files.download("max_depth_"+str(rf_all)) '''